<center>
<h1><b>Homework 5</b></h1>
<h2>PHYS 580 - Computational Physics</h2>
<h4>Professor Molnar</h4>
</br>
<h5><b>Ethan Knox</b></h5>
<h6>https://www.github.com/ethank5149</h6>
<h6>ethank5149@gmail.com</h6>
</br>
<h5><b>November 14, 2020</b></h5>
</center>
<hr>

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
from numpy.polynomial import Polynomial
from scipy.optimize import curve_fit
from tqdm.notebook import trange, tqdm
from ipywidgets import fixed, interact_manual, IntSlider, FloatSlider, FloatRangeSlider, ToggleButton

%run ClusterGrowth.py
%run IsingModel.py

### Settings

In [2]:
plt.style.use('dark_background')
pd.set_option('use_inf_as_na', True)
pd.options.mode.chained_assignment = None

### Globals

In [3]:
Tc = 2. / np.log(1. + np.sqrt(2.))
linfit = lambda x, c0, c1 : c0 + c1 * x

# Problem 1
## 7.30 (p.228)

Generate a spanning cluster for a two dimensional square lattice at $p=p_c$ using any of the search methods discussed in connection with Figure 7.29. Estimate the fractal dimensionality of the cluster. You should find a value of $d_f$, which is slightly smaller than $2$ (the expected value is $91/48\approx1.90$).

In [4]:
@interact_manual(
    nrows=IntSlider(value=200,min=100,max=500,step=10,description=r'$n_{rows}$'), 
    ncols=IntSlider(value=200,min=100,max=500,step=10,description=r'$n_{cols}$'), 
    fill=FloatSlider(value=0.15,min=0.01,max=0.25,step=0.01,description='% Fill'))
def problem1(nrows, ncols, fill):
    p = ClusterGrowth(nrows, ncols, fill)
    df = p.simulate()
        
    df['logR'] = df.Radius.apply(np.log)
    df['logM'] = df.Mass.apply(np.log)
    df['x'] = df.logR.where(df.Radius > 0.0).where(df.Radius <= p.pseudo_radius())
    df['y'] = df.logM.where(df.Mass > 0.0)
    df['dydx'] = df.y.diff().div(df.x.diff())
    df = df.dropna()
    
    (c, beta), _ = curve_fit(lambda _, c, beta : c + beta * _, df.x.values, df.y.values)
    
    fit = Polynomial.fit(df.x.values, df.y.values, 1)
    df['fit'] = df.x.apply(fit)
    d_f = df.fit.diff().div(df.x.diff()).mean()
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(32,16), dpi=200)   
    ax1.set_title(r'Cluster Growth - Eden Model')
    ax1.matshow(p.grid) 
    
    df.plot.scatter('x', 'y', ax=ax2, label='Simulation')
    df.plot('x', 'fit', ls='--', ax=ax2, label=rf'Model $\left(d_f\approx{beta:0.4f}\right)$')
    ax2.plot(df.x.values, np.log(2. * np.pi) + 91./48. * df.x.values , label=rf'Analytic $\left(d_f={91./48.:0.4f}\right)$')
    ax2.plot(df.x.values, np.log(2. * np.pi) + 2. * df.x.values   , label=rf'Reference $\left(d_f={2.:0.4f}\right)$')
    
    ax2.set_xlabel(r'$\ln(r)$')
    ax2.set_ylabel(r'$\ln(m)$')
    ax2.legend(prop={'size': 24})
    ax2.grid()
    plt.savefig('plots/problem1')

interactive(children=(IntSlider(value=200, description='$n_{rows}$', max=500, min=100, step=10), IntSlider(val…

<img src="plots/problem1.png">

# Problem 2
## 8.3 (p.257)

Calculate $M$ for the Ising model on a square lattice and try to estimate $\beta$. You should find a value close to $1/8$. Repeat this calculation for a triangular lattice. It turns out that $\beta$ is the same for all regular two dimensional lattices. However, its value does depend on the dimensionality, as studied in the next problem.

_Hint:_ You should find that the power law (8.17) with $\beta\approx1/8$ is obeyed reasonably well for $2.0<T<T_c\approx2.27$.

It is enough if you calculate __either__ for the square grid, __or__ the triangular one (the latter takes a little more thought). Doing both cases is optional.

In [5]:
@interact_manual(
    ncols=IntSlider(value=10,min=5,max=25,step=5,description=r'$n_{rows}$'), 
    nrows=IntSlider(value=10,min=5,max=25,step=5,description=r'$n_{cols}$'), 
    J=fixed(1.), 
    kB=fixed(1.), 
    Trange=FloatRangeSlider(value=[2., Tc], min=1.5, max=2.5,step=0.05, description=r'$\left(T_i,T_f\right)$'),
    nT=IntSlider(value=100,min=100,max=1000,step=100,description=r'$n_T$'), 
    Hrange=fixed([0.0, 1.0]),
    nH=fixed(1), 
    nR=IntSlider(value=100,min=100,max=1000,step=100,description=r'$n_R$'), 
    nS=IntSlider(value=1000,min=100,max=1000,step=100,description=r'$n_S$'))
def problem2(ncols, nrows, J, kB, Trange, nT, Hrange, nH, nR, nS):
    model = IsingModel(ncols, nrows, J, kB, Trange, nT, Hrange, nH, nR, nS)
    df = model.simulate();
        
    fig, ax = plt.subplots(2, 2, figsize=(16,9), dpi=300)
    fig.suptitle('Ising Model (Metropolis-Hastings Algorithm)\nSimulation Results From Problem 2')
    df.plot.scatter('Temperature', 'Energy'                , ax=ax[0,0])
    df.plot.scatter('Temperature', 'Magnetization'         , ax=ax[0,1])
    df.plot.scatter('Temperature', 'SpecificHeatCapacity'  , ax=ax[1,0])
    df.plot.scatter('Temperature', 'MagneticSusceptibility', ax=ax[1,1])
    fig.savefig('plots/problem2_simulation_results')
    plt.close()
    
    df['x'] = df.Temperature.apply(lambda _ : np.log(np.abs(_ - Tc) / 1. + 0*Tc))
    df['y'] = df.SpecificHeatCapacity.apply(lambda _ : np.log(np.abs(_)))
    df = df.dropna()
    
    (c, minus_beta), _ = curve_fit(linfit, df.x.values, df.y.values)
    df['fit'] = df.x.apply(lambda _ : linfit(_, c, minus_beta))
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 9), dpi=300)
    ax.set_title('Ising Model (Metropolis-Hastings Algorithm)\nCalculation Of Critical Exponent ' + r'$\beta$')
    df.plot('x', 'fit', ax=ax, label=fr'$y\propto-\beta x$, $\beta={-minus_beta:0.4f}$')
    df.plot.scatter('x', 'y', ax=ax, label='Simulation')
    
    ax.legend()
    fig.savefig('plots/problem2')

interactive(children=(IntSlider(value=10, description='$n_{rows}$', max=25, min=5, step=5), IntSlider(value=10…

<img src="plots/problem2.png"> 

# Problem 3
## 8.7 (p.258)

Obtain the specific heat as a function of temperature for a $10\times10$ square lattice by differentiating the energy and through the fluctuation-dissipation theorem. Show that the two methods give the same result. Which approach is more accurate (for a given amount of computer time)?

In [6]:
@interact_manual(
    ncols=IntSlider(value=20,min=10,max=100,step=10,description=r'$n_{rows}$'), 
    nrows=IntSlider(value=20,min=10,max=100,step=10,description=r'$n_{cols}$'), 
    J=fixed(1.), 
    kB=fixed(1.), 
    Trange=FloatRangeSlider(value=[1.5, 3.0], min=1.5, max=3.0, step=0.1, description=r'$\left(T_i,T_f\right)$'),
    nT=IntSlider(value=100,min=100,max=1000,step=100,description=r'$n_T$'), 
    Hrange=fixed([0.0, 1.0]),
    nH=fixed(1), 
    nR=IntSlider(value=1000,min=100,max=1000,step=100,description=r'$n_R$'), 
    nS=IntSlider(value=1000,min=100,max=1000,step=100,description=r'$n_S$'))
def problem3(ncols, nrows, J, kB, Trange, nT, Hrange, nH, nR, nS):
    model = IsingModel(nrows, ncols, J, kB, Trange, nT, Hrange, nH, nR, nS)
    df = model.simulate();
    df['tau'] = df.Temperature.apply(lambda _ :  (_ - Tc) / 1. + 0*Tc)
    df['dEdT'] = df.Energy.diff().div(df.tau.diff()).abs()
    df['SpecificHeatCapacityAlt'] = df.dEdT.div(df.tau.pow(2.))
    df['SpecificHeatCapacityAlt'] = df.SpecificHeatCapacityAlt.where(df.SpecificHeatCapacityAlt <= df.SpecificHeatCapacityAlt.mean())
    
    fig, ax = plt.subplots(2, 2, figsize=(16,9), dpi=300)
    fig.suptitle('Ising Model (Metropolis-Hastings Algorithm)\nSimulation Results From Problem 3')
    df.plot.scatter('tau', 'Energy', ax=ax[0, 0])
    df.plot.scatter('tau', 'Magnetization', ax=ax[0, 1])
    df.plot.scatter('tau', 'SpecificHeatCapacity', ax=ax[1, 0])
    df.plot.scatter('tau', 'MagneticSusceptibility', ax=ax[1, 1])
    fig.savefig('plots/problem3_simulation_results')
    plt.close()
    
    fig, ax = plt.subplots(2, 1, figsize=(16,9), dpi=300, sharex=True)
    fig.suptitle('Ising Model (Metropolis-Hastings Algorithm)\nSpecific Heat Capacity vs. Temperature')
    df.plot.scatter('tau', 'SpecificHeatCapacity', c='darkred', ax=ax[0], label=r'Fluctuation Dissipation Theorem: $C(T)=\frac{Var(E)}{k_BT^2}$')
    df.plot.scatter('tau', 'SpecificHeatCapacityAlt', c='darkgreen', ax=ax[1], label=r'Thermodynamics: $C(T)=\frac{dE(T)}{dT}$')
    fig.savefig('plots/problem3')

interactive(children=(IntSlider(value=20, description='$n_{rows}$', min=10, step=10), IntSlider(value=20, desc…

<img src="plots/problem3.png"> 

# Problem 4
## 8.15 (p.267)

Scaling behavior is found for thermodynamic quantities other than the magnetization. Calculate the susceptibility $\chi$ at various values of $T$ and $H$ around the critical point of the Ising model on a square lattice, and study data collapsing using your results. The scaling form for $\chi$ is 
$$\chi(t, h)=|t|^{-\gamma}g_{\pm}\left(\frac{h}{|t|^{\beta\delta}}\right),$$
where the critical exponent $\gamma=7/4$.

In [7]:
@interact_manual(
    ncols=IntSlider(value=15,min=5,max=25,step=5,description=r'$n_{rows}$'), 
    nrows=IntSlider(value=15,min=5,max=25,step=5,description=r'$n_{cols}$'), 
    J=fixed(1.), 
    kB=fixed(1.),
    Trange=FloatRangeSlider(value=[2., 2.5], min=1.5, max=3.0,step=0.01, description=r'$\left(T_i,T_f\right)$'),
    nT=IntSlider(value=100,min=10,max=250, step=10,description=r'$n_T$'), 
    Hrange=FloatRangeSlider(value=[-1., 1.], min=-10., max=10.,step=0.01, description=r'$\left(H_i,H_f\right)$'),
    nH=IntSlider(value=100,min=10,max=250,step=10,description=r'$n_H$'), 
    nR=IntSlider(value=250,min=200,max=1000,step=100,description=r'$n_R$'), 
    nS=IntSlider(value=1000,min=200,max=1000,step=100,description=r'$n_S$'))
def problem4(ncols, nrows, J, kB, Trange, nT, Hrange, nH, nR, nS):
    model  = IsingModel(ncols, nrows, J, kB, Trange, nT, Hrange, nH, nR, nS)
    df = model.simulate();
    df['tau'] = df.Temperature.apply(lambda _ :  (_ - Tc) / Tc)
    
    fig = plt.figure(figsize=(16,16), dpi=300, constrained_layout=True)
    fig.suptitle('Ising Model (Metropolis-Hastings Algorithm)\nSimulation Results From Problem 4')

    ax1 = fig.add_subplot(221, projection='3d')
    ax1.view_init(30, 120)
    ax1.plot_trisurf(df.MagneticField, df.tau, df.Energy, cmap=plt.get_cmap('jet'))
    ax1.set_xlabel('Magnetic Field')
    ax1.set_ylabel(r'$\tau$')
    ax1.set_zlabel('Energy')

    ax2 = fig.add_subplot(222, projection='3d')
    ax2.view_init(30, 70)
    ax2.plot_trisurf(df.MagneticField, df.tau, df.Magnetization, cmap=plt.get_cmap('jet'))
    ax2.set_xlabel('Magnetic Field')
    ax2.set_ylabel(r'$\tau$')
    ax2.set_zlabel('Magnetization')

    ax3 = fig.add_subplot(223, projection='3d')
    ax3.view_init(30, -110)
    ax3.plot_trisurf(df.MagneticField, df.tau, df.SpecificHeatCapacity, cmap=plt.get_cmap('jet'))
    ax3.set_xlabel('Magnetic Field')
    ax3.set_ylabel(r'$\tau$')
    ax3.set_zlabel('Specific Heat Capacity')

    ax4 = fig.add_subplot(224, projection='3d')
    ax4.view_init(30, -110)
    ax4.plot_trisurf(df.MagneticField, df.tau, df.MagneticSusceptibility, cmap=plt.get_cmap('jet'))
    ax4.set_xlabel('Magnetic Field')
    ax4.set_ylabel(r'$\tau$')
    ax4.set_zlabel('Magnetic Susceptibility')

    fig.savefig('plots/problem4')

interactive(children=(IntSlider(value=15, description='$n_{rows}$', max=25, min=5, step=5), IntSlider(value=15…

<img src="plots/problem4.png">